# 07: Neural Networks II
Useful resources:
- [Visual Analytics in Deep Learning: An Interrogative Survey for the Next Frontiers](https://arxiv.org/pdf/1801.06889.pdf)
- [Distill](https://distill.pub)

## Imports

In [ ]:
from dataclasses import dataclass, field
from itertools import product
import random

import altair as alt
import numpy as np
import pandas as pd
import pmlb

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.manifold import TSNE

In [ ]:
# If you're running this code locally, then you can uncomment this to automatically
# save the chart data in files, rather than including the data in the spec. 

# !mkdir -p data
# alt.data_transformers.enable('json', prefix='data/altair-data')

## Data Preparation and Modeling

Load the mnist dataset and take a random sample of it.

In [ ]:
mnist = pmlb.fetch_data('mnist')

In [ ]:
mnist_small = mnist.sample(n=30000)

Separate the feature values from the target labels. Split the dataset into train and test sets.

In [ ]:
X = mnist_small.drop(columns=['target']).values
y = mnist_small['target'].values

In [ ]:
classes = sorted(list(set(y)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

Next we'll train a multi-layer perceptron on this dataset.

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=(512, 256))
nn.fit(X_train, y_train)

In [ ]:
nn.score(X_test, y_test)

In [ ]:
y_pred = nn.predict(X_test)

## Helper functions

This model has 4 layers: an input layer, two hidden layers, and an output layer. The hidden layers use the ReLU activation function. The output layer uses the softmax function.

In [ ]:
nn.n_layers_

The `get_layer_output` function below returns the output of the model at a given layer.

References:
- [sklearn source code for generating model predictions](https://github.com/scikit-learn/scikit-learn/blob/f3f51f9b611bf873bd5836748647221480071a87/sklearn/neural_network/_multilayer_perceptron.py#L144).
- [sklearn source code for ReLU and softmax activations](https://github.com/scikit-learn/scikit-learn/blob/f3f51f9b611bf873bd5836748647221480071a87/sklearn/neural_network/_base.py#L47)
- [scipy source code for softmax](https://github.com/scipy/scipy/blob/v1.9.3/scipy/special/_logsumexp.py#L130-L223)

In [ ]:
def relu(X):
    return np.maximum(X, 0)

def softmax(X):
    return np.exp(X) / np.exp(X).sum(axis=1, keepdims=True)

def get_layer_output(model, X, layer):
    output = X
    
    for i in range(layer - 1):
        z = np.dot(output, model.coefs_[i]) + model.intercepts_[i]
        
        if i < model.n_layers_ - 2:
            output = relu(z)
        else:
            output = softmax(z)
        
    return output

For example, we can see that getting the output of the last layer is the same as calling the model's `predict_proba` function.

In [ ]:
nn.predict_proba(X_train[0:3])

In [ ]:
get_layer_output(nn, X_train[0:3], nn.n_layers_)

The `get_df` function below transforms a numpy array representing an image into a dataframe.

In [ ]:
def get_df(data):
    indices = np.arange(data.shape[0])
    size = int(np.sqrt(data.shape[0]))
    x = indices % size
    y = np.floor(indices / size)
    
    return pd.DataFrame({
        'x': x,
        'y': y,
        'value': data
    })

In [ ]:
get_df(X_train[0])

## Neuron Activation Matrix

The [ActiVis paper](https://arxiv.org/pdf/1704.01942.pdf) contains a neuron activation matrix. Each row in the matrix represents a subset of instances. Each column represents a neuron in the neural network. Let's create a version of this matrix for our model. We will subset the instances by their true class label.

**Exercise 1:**

First, we need a function that computes the average activation for each hidden neuron in our model for a given set of instances. This function should return a flat list of the average activations for the neurons in the hidden layers. In our case, there are two hidden layers with a combined total of 768 neurons, so this list should contain 768 numbers.

In [ ]:
'''
X - 2D numpy array representing a set of instances
nn - sklearn MLPClassifier
'''
def get_average_activations(X, nn):
    activations = []
    
    for layer in range(2, nn.n_layers_):
        activations.extend(get_layer_output(nn, X, layer).mean(axis=0))

    return activations

In [ ]:
get_average_activations(X_test[y_test == 0], nn)

**Exercise 2:**

Next, we need a function that puts the average activations for each subset of instances into a single dataframe. We will split the instances into subsets based on their ground-truth label. Each row of the dataframe will represent a pair of an instance subset and a neuron (i.e. each row is one cell in the matrix). The dataframe will have three columns:
- "neuron" for the index of the neuron
- "label" for the label of the instances in the subset
- "activation" for the average activation of the neuron for the instances in the subset

In [ ]:
'''
X - 2D numpy array containing the instances
y - 1D numpy array containing the ground-truth labels
nn - sklearn MLPClassifier
'''
def calculate_activation_matrix(X, y, nn):
    dfs = []
    
    # 2a: get a list of the unique labels
    classes = sorted(list(set(y)))
    
    # 2b: we'll refer to the neurons by their index in the lists returned
    # by get_average_activations. for convenience, we'll get the list of
    # neuron indices here (i.e. [0, 1, 2, 3, ..., 766, 767])
    num_neurons = sum(nn.hidden_layer_sizes)
    neuron_ids = list(range(num_neurons))
    
    for label in classes:
        # 2c: get the average activation for the instances with this label
        activations = get_average_activations(X[y == label], nn)
        
        # 2d: create a dataframe for this subset
        df = pd.DataFrame({
            'neuron': neuron_ids,
            'label': label,
            'activation': activations
        })
        
        dfs.append(df)
        
    return pd.concat(dfs)

In [ ]:
activations = calculate_activation_matrix(X_test, y_test, nn)

**Exercise 3:**

768 neurons is too many to show at once. In the ActiVis paper, you can choose the neurons that have the highest activations for a given class. Complete the `get_top_neurons` function below. It should return a sorted list of the top `num_neurons` neurons that have the highest average activation for the instances with the given `label`.

In [ ]:
'''
activations - pandas dataframe
num_neurons - number of neurons to take
label - subset of instances to sort the neurons by
'''
def get_top_neurons(activations, num_neurons, label):
    # filter the df to only contain subsets with the given label
    activations = activations[activations["label"] == label]
    mean_by_neuron = activations.groupby('neuron').mean()
    top_neurons = mean_by_neuron.sort_values('activation', ascending=False).index[0:num_neurons]
    return list(top_neurons)

In [ ]:
num_neurons = 40

In [ ]:
top_neurons_0 = get_top_neurons(activations, num_neurons, 0)
top_neurons_0

**Exercise 4:**

Once we've computed a sorted list of `neurons`, we need a function that will filter the `activations` dataframe to only contain those neurons.

In [ ]:
'''
activations - pandas dataframe
neurons - list containing indices of neurons
'''
def filter_activations(activations, neurons):
    return activations[activations['neuron'].isin(neurons)]

In [ ]:
filter_activations(activations, top_neurons_0)

**Exercise 5:**

Complete the `neuron_activation_matrix` function below. It should return a heatmap of the activations for the top `num_neurons` for the given `label`. You should call `get_top_neurons` and `filter_activations` in this function.

Note: the heatmap will still have one row per label. The `label` argument determines which subset of instances is used to sort the neurons.

In [ ]:
'''
activations - pandas dataframe
num_neurons - number of neurons (columns) to show in the matrix
label - subset of instances to sort the neurons by
'''
def neuron_activation_matrix(activations, num_neurons, label):
    neurons = get_top_neurons(activations, num_neurons, label)
    activations = filter_activations(activations, neurons)
    return alt.Chart(activations).mark_rect().encode(
        x=alt.X('neuron:N', sort=neurons),
        y='label:N',
        color='activation'
    )

In [ ]:
neuron_activation_matrix(activations, num_neurons, 0)

**Exercise 6:**

Concatenate the neuron activation matrices sorted by each label.

In [ ]:
'''
activations - pandas dataframe
num_neurons - number of columns in each matrix
labels - list of numbers
'''
def all_neuron_activation_matrices(activations, num_neurons, labels):
    charts = [neuron_activation_matrix(activations, num_neurons, label) for label in labels]
    return alt.vconcat(*charts)

In [ ]:
all_neuron_activation_matrices(activations, num_neurons, range(10))